### 목표 : 생선 분류 모델
- 데이터 : fish.csv
- 피처 : 5개 (Weight, Length, Diagonal, Height, Width)
- 타겟 : 1개 (Species)
- 방법 : 지도학습 + 다중분류

(1) 모듈 로딩 및 데이터 준비 <hr>

In [258]:
# 모듈로딩
import pandas as pd
import numpy as np

#여기서는 그냥 logistic 이랑 ova, ovr 볼거라서 전처리는 생략하겠긔

In [259]:
# 데이터 준비
file = '../data/fish.csv'
fishDF = pd.read_csv(file)
fishDF.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


(2) 학습 위한 데이터 준비  <hr>

(2-1) 피처/타겟 분리

In [260]:
featureDF = fishDF[fishDF.columns[1:]]
targetDF = fishDF[fishDF.columns[0]]

In [261]:
print(f"featureDF : {featureDF.shape}")
print(f"targetDF : {targetDF.shape}")

featureDF : (159, 5)
targetDF : (159,)


In [262]:
# 타겟의 클래스 수 확인
targetDF.nunique()

7

In [263]:
# 타겟 클래스 별 데이터 수 확인
targetDF.value_counts()

# target class별 데이터 수 확인
(targetDF.value_counts() / targetDF.shape[0]) * 100

Species
Perch        35.220126
Bream        22.012579
Roach        12.578616
Pike         10.691824
Smelt         8.805031
Parkki        6.918239
Whitefish     3.773585
Name: count, dtype: float64

In [264]:
# 이 방법으로 숫자로 바꾸거나 아니면 replace 하면 됨
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(targetDF)
targetDF = encoder.transform(targetDF)
targetDF

# inverse_transform => 반대로 바꿔줌. 원래대로 하는 거 ㅇㅇ

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5])

(2-2) 학습 / 테스트용  데이터셋  준비

In [265]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, stratify=targetDF, random_state=11)
print(f'[train dataset] {X_train.shape}, {y_train.shape}')
print(f'[test dataset] {X_test.shape}, {y_test.shape}')

[train dataset] (119, 5), (119,)
[test dataset] (40, 5), (40,)


(3) 학습 진행

In [266]:
from sklearn.linear_model import LogisticRegression

In [267]:
# 모델 인스턴스 생성 및 학습
model = LogisticRegression(max_iter = 20000, solver = 'liblinear')    #solver라는 파라미터 설정을 통해 튜닝을 해서 test score를 올렸다. 
#max_iter는 전체 데이터를 몇번 공부하겠냐 ? 즉, 몇 번 회독하겠냐?
#solver는 공부방식을 바꾸는거임 
#solver –  default='lbfgs'
#   Algorithm to use in the optimization problem. Default is 'lbfgs'. To choose a solver, you might want to consider the following aspects:
#   For small datasets, 'liblinear' is a good choice, whereas 'sag' and 'saga' are faster for large ones;
#   For multiclass problems, only 'newton-cg', 'sag', 'saga' and 'lbfgs' handle multinomial loss;
#   'liblinear' is limited to one-versus-rest schemes.
model.fit(X_train, y_train)

LogisticRegression(max_iter=20000, solver='liblinear')

In [268]:
#모델 파라미터 확인
print(f"classes_: {model.classes_}")
print(f"feature_names_in_ : {model.feature_names_in_}")
print(f"max_iter : {model.max_iter}")
print(f"coef_ : {len(model.coef_)}개 \n {model.coef_}")

# 왜 coef_ 가 7개 나올까 ? => target 셋에서 종류가 7개임

classes_: [0 1 2 3 4 5 6]
feature_names_in_ : ['Weight' 'Length' 'Diagonal' 'Height' 'Width']
max_iter : 20000
coef_ : 7개 
 [[ 1.31151754e-02 -1.64944470e+00  8.28009575e-01  1.41621595e+00
  -4.15067201e-01]
 [-2.10617657e-02  3.33701594e-01 -9.64909143e-01  2.19381184e+00
   2.66611701e-02]
 [-1.97453974e-03  2.60616873e+00 -2.66412260e+00 -7.93176743e-03
   1.91659551e+00]
 [ 1.01422059e-02  2.55168743e-01  1.51461260e-01 -1.94779290e+00
  -8.36602128e-01]
 [-9.89829706e-03 -1.72578825e+00  1.53807538e+00 -5.12880032e-01
   1.65750894e+00]
 [-7.29426634e-02  3.82049401e-01  1.62783679e-01 -1.55364795e+00
  -5.97839461e-01]
 [ 5.68775586e-03 -5.20399292e-01  2.54546484e-01 -2.46921990e-01
   8.40269158e-01]]


(4) 평가 <hr>

In [269]:
print(f"[Train Score] {model.score(X_train, y_train)}")
print(f"[Test Score] {model.score(X_test, y_test)}")

[Train Score] 0.9495798319327731
[Test Score] 0.975


(5) 모델 활용 <hr>

In [270]:
y_pre = model.predict(X_test.iloc[[0]])
y_pre

array([0])

In [271]:
y_test.iloc[0]

# 위의 예측값과 실제 데이터를 확인해보니 일치한다! 무야홍~

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [ ]:
model.predict_proba(X_test.iloc[[0]])

# 아까 7개의 모델이 있었다. 그걸 각각 돌려봤을 때 어떤 확률로 각각의 종류가 나왔을까 => 그래서 젤 높은 확률로 나온게 Bream이라는 종류였다

In [272]:
# 5개 데이터에 대한 생선 분류 예측
print(model.classes_)
np.round(model.predict_proba(X_test.iloc[:5]), 3)

# 1개의 행을 뽑아서 다 더하면 1이 된다
# 여기서 행 1개가 원본 데이터 1개가 된다. 원본데이터 1개에서 각 생선종류별로 어떤 확률이 될지 나온다
# 그래서 젤 높은 확률이 되는것이 원본데이터 1개의 예측 생선 종류이다

[0 1 2 3 4 5 6]


array([[0.504, 0.311, 0.   , 0.   , 0.173, 0.   , 0.012],
       [0.158, 0.73 , 0.044, 0.   , 0.057, 0.   , 0.01 ],
       [0.772, 0.024, 0.001, 0.   , 0.18 , 0.   , 0.023],
       [0.001, 0.089, 0.719, 0.002, 0.155, 0.004, 0.03 ],
       [0.   , 0.021, 0.753, 0.009, 0.176, 0.009, 0.031]])

In [273]:
y_test[:5].to_list()

AttributeError: 'numpy.ndarray' object has no attribute 'to_list'

In [274]:
result = model.predict_proba(X_test.iloc[:5]).argmax(axis = 1)
result

array([0, 1, 0, 2, 2], dtype=int64)

In [275]:
data = {"Pre Y":[model.classes_[idx] for idx in result],
        "True Y": y_test[:5].to_list()}

AttributeError: 'numpy.ndarray' object has no attribute 'to_list'

In [276]:
pd.DataFrame(data)

# 예측값과 진짜값이 동일하게 나온걸 확인할 수 있다.

,Pre Y,True Y
0,Bream,Bream
1,Parkki,Parkki
2,Bream,Bream
3,Perch,Perch
4,Perch,Perch


(6) 모델 성능 평가 <hr>
- 정확도
- 정밀도
- 재현율
- F1 - score
- Confusion Matrics
- Classification Report

In [277]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

In [278]:
X_test.shape

(40, 5)

In [279]:
result = classification_report(y_test, model.predict(X_test), zero_division = 0)
print(result)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00         3
           2       0.93      1.00      0.97        14
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         4
           6       0.00      0.00      0.00         1

    accuracy                           0.97        40
   macro avg       0.85      0.86      0.85        40
weighted avg       0.95      0.97      0.96        40


In [282]:
f1_score(y_test, model.predict(X_test), average = 'macro')

0.852216748768473

In [283]:
f1_score(y_test, model.predict(X_test), average = 'weighted')

0.9629310344827587

In [285]:
recall_score(y_test, model.predict(X_test), average = 'micro')

0.975

In [286]:
confusion_matrix(y_test, model.predict(X_test), )

array([[ 9,  0,  0,  0,  0,  0,  0],
       [ 0,  3,  0,  0,  0,  0,  0],
       [ 0,  0, 14,  0,  0,  0,  0],
       [ 0,  0,  0,  4,  0,  0,  0],
       [ 0,  0,  0,  0,  5,  0,  0],
       [ 0,  0,  0,  0,  0,  4,  0],
       [ 0,  0,  1,  0,  0,  0,  0]], dtype=int64)